# Part A - Van Gogh Painting Classifier

## Deep Learning Project - Tel Aviv University

This notebook implements a binary classifier to identify Van Gogh paintings using transfer learning with VGG19.

### Overview:
1. **Load Data**: Read pre-prepared CSV file with image paths and labels
2. **Split Dataset**: 70% train, 15% validation, 15% test
3. **Data Augmentation**: Apply transforms for training robustness
4. **Model**: VGG19 pre-trained on ImageNet, fine-tuned for binary classification
5. **Hyperparameter Tuning**: Use Optuna to find best parameters
6. **Training**: Train final model with best hyperparameters
7. **Evaluation**: Test set metrics and visualizations

### Requirements:
- Google Colab with GPU runtime
- `post_impressionism_data.csv` file (created by Get_Post_Impressionism_Data.ipynb)
- Weights & Biases account for experiment tracking

---


## 1. Environment Setup


In [ ]:
# Check if running on Google Colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("Running on Google Colab")
else:
    print("Running locally")


Running locally


In [2]:
# Install required packages: optuna (hyperparameter tuning), wandb (experiment tracking)
%pip install -q optuna wandb


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
Note: you may need to restart the kernel to use updated packages.


## 2. Import Libraries


In [ ]:
# Import libraries
import os
import platform
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from tqdm import tqdm

# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score,
    precision_score, recall_score, f1_score, roc_auc_score, roc_curve
)

# Hyperparameter tuning and logging
import optuna
import wandb

# Set random seeds for reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

# Hardware Benchmark Setup (Required by instructions)
print("="*60)
print("HARDWARE BENCHMARK INFORMATION")
print("="*60)
print(f"Python version: {platform.python_version()}")
print(f"Machine name (node): {platform.node()}")
print(f"Current working directory: {os.getcwd()}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory Total: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    print(f"GPU Memory Allocated: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
    print(f"GPU Memory Cached: {torch.cuda.memory_reserved(0) / 1024**3:.2f} GB")
print(f"PyTorch version: {torch.__version__}")
print("="*60)


In [ ]:
# Setup device (GPU/CPU) - GPU is much faster for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

if device.type == 'cpu':
    print("WARNING: Running on CPU. Enable GPU in Colab: Runtime -> Change runtime type -> GPU")


In [ ]:
# CPU vs GPU Benchmark (Required by instruction 1.34)
# Measure time for 20 forward+backward iterations on both CPU and GPU
import time

print("="*60)
print("CPU vs GPU BENCHMARK")
print("="*60)
print(f"Machine name: {platform.node()}")
if torch.cuda.is_available():
    print(f"GPU model: {torch.cuda.get_device_name(0)}")
print("="*60)

# Create a dummy model for benchmarking
dummy_model = nn.Sequential(
    nn.Conv2d(3, 64, 3, padding=1),
    nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
    nn.Flatten(),
    nn.Linear(64, 2)
)
dummy_input = torch.randn(8, 3, 224, 224)
criterion = nn.CrossEntropyLoss()
dummy_labels = torch.randint(0, 2, (8,))

# CPU Benchmark
print("\nRunning CPU benchmark (20 iterations)...")
cpu_model = dummy_model.to('cpu')
cpu_input = dummy_input.to('cpu')
cpu_labels = dummy_labels.to('cpu')
cpu_optimizer = torch.optim.Adam(cpu_model.parameters(), lr=0.001)

start_time = time.time()
for _ in range(20):
    cpu_optimizer.zero_grad()
    outputs = cpu_model(cpu_input)
    loss = criterion(outputs, cpu_labels)
    loss.backward()
    cpu_optimizer.step()
cpu_time = time.time() - start_time

print(f"CPU time: {cpu_time:.4f} seconds ({cpu_time/20*1000:.2f} ms per iteration)")

# GPU Benchmark (if available)
if torch.cuda.is_available():
    print("\nRunning GPU benchmark (20 iterations)...")
    gpu_model = dummy_model.to('cuda')
    gpu_input = dummy_input.to('cuda')
    gpu_labels = dummy_labels.to('cuda')
    gpu_optimizer = torch.optim.Adam(gpu_model.parameters(), lr=0.001)
    
    # Warmup
    for _ in range(5):
        gpu_optimizer.zero_grad()
        outputs = gpu_model(gpu_input)
        loss = criterion(outputs, gpu_labels)
        loss.backward()
        gpu_optimizer.step()
    torch.cuda.synchronize()
    
    start_time = time.time()
    for _ in range(20):
        gpu_optimizer.zero_grad()
        outputs = gpu_model(gpu_input)
        loss = criterion(outputs, gpu_labels)
        loss.backward()
        gpu_optimizer.step()
    torch.cuda.synchronize()
    gpu_time = time.time() - start_time
    
    print(f"GPU time: {gpu_time:.4f} seconds ({gpu_time/20*1000:.2f} ms per iteration)")
    print(f"\nSpeedup: {cpu_time/gpu_time:.2f}x faster on GPU")
else:
    print("\nGPU not available, skipping GPU benchmark")

print("="*60)

In [ ]:
# Login to Weights & Biases for experiment tracking
wandb.login(key="16d1bc863b28f81253ac0ee253b453393791a7e1")
print("Logged in to Weights & Biases")


## 3. Data Preparation


In [ ]:
# Scan Post_Impressionism directory and create metadata CSV
import os
import pandas as pd

# Find Post_Impressionism directory (works in both Kaggle and Colab)
possible_dirs = [
    "/kaggle/input/wikiart/Post_Impressionism",  # Kaggle
    "/content/data/Post_Impressionism",          # Colab
    "/content/Post_Impressionism",               # Colab alternative
    "/content/wikiart/Post_Impressionism",       # Colab alternative
]

base_dir = None
for dir_path in possible_dirs:
    if os.path.exists(dir_path):
        base_dir = dir_path
        break

if base_dir is None:
    raise FileNotFoundError(
        f"Post_Impressionism directory not found!\n"
        f"   Checked: {possible_dirs}\n"
        f"   Please download images to one of these locations."
    )

print(f"Found images in: {base_dir}")

# Scan directory and create DataFrame (like Nir)
records = []
for fname in os.listdir(base_dir):
    if not fname.lower().endswith((".jpg", ".png")):
        continue
    
    artist = fname.split("_")[0]  # Extract artist name from filename
    
    records.append({
        "filepath": os.path.join(base_dir, fname),
        "filename": fname,
        "artist": artist,
        "is_van_gogh": 1 if "van-gogh" in artist.lower() else 0
    })

df = pd.DataFrame(records)

# Save metadata to CSV
csv_output_path = "/kaggle/working/post_impressionism_data.csv" if os.path.exists("/kaggle") else ("/content/post_impressionism_data.csv" if IN_COLAB else "post_impressionism_data.csv")
df.to_csv(csv_output_path, index=False)
print(f"Saved metadata CSV: {csv_output_path}")

print(f"\nLoaded: {len(df)} images")
print(f"  Van Gogh: {df['is_van_gogh'].sum()}")
print(f"  Other: {len(df) - df['is_van_gogh'].sum()}")


In [ ]:
# Split dataset: 70% train, 15% validation, 15% test (stratified to maintain class balance)
train_df, temp_df = train_test_split(
    df, test_size=0.3, stratify=df["is_van_gogh"], random_state=SEED
)

val_df, test_df = train_test_split(
    temp_df, test_size=0.5, stratify=temp_df["is_van_gogh"], random_state=SEED
)

print("Dataset splits:")
print(f"  Train: {len(train_df):5d} ({len(train_df)/len(df):.1%})")
print(f"  Val:   {len(val_df):5d} ({len(val_df)/len(df):.1%})")
print(f"  Test:  {len(test_df):5d} ({len(test_df)/len(df):.1%})")

print("\nClass distribution:")
print(f"  Train - Van Gogh: {train_df['is_van_gogh'].mean():.2%}")
print(f"  Val   - Van Gogh: {val_df['is_van_gogh'].mean():.2%}")
print(f"  Test  - Van Gogh: {test_df['is_van_gogh'].mean():.2%}")

# Save dataset splits for later use (so data preparation doesn't need to be rerun)
import pickle
import os
splits_save_path = 'dataset_splits.pkl'
with open(splits_save_path, 'wb') as f:
    pickle.dump({
        'train_df': train_df,
        'val_df': val_df,
        'test_df': test_df
    }, f)
print(f"\nDataset splits saved to '{splits_save_path}' (can skip data preparation next time)")


## 4. Data Transforms & Dataset Class


In [ ]:
# Define image transforms: resize to 224x224, normalize with ImageNet stats
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

# Training: data augmentation (random crop, flip, rotation, color jitter)
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

# Evaluation: no augmentation (consistent evaluation)
eval_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

print("Transforms defined")


In [ ]:
# Custom PyTorch Dataset class to load images from file paths
class VanGoghDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform
        # Filter out files that don't exist
        self.df = self.df[self.df['filepath'].apply(os.path.exists)].reset_index(drop=True)
        if len(self.df) < len(df):
            print(f"Warning: {len(df) - len(self.df)} files not found and removed")

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        try:
            image = Image.open(row["filepath"]).convert("RGB")
            label = row["is_van_gogh"]
            if self.transform:
                image = self.transform(image)
            return image, label
        except Exception as e:
            print(f"Error loading {row['filepath']}: {e}")
            # Return a black image as fallback (shouldn't happen if we filtered)
            image = Image.new('RGB', (224, 224), color='black')
            label = row["is_van_gogh"]
            if self.transform:
                image = self.transform(image)
            return image, label

# Load dataset splits if they don't exist (in case data preparation wasn't run)
if 'train_df' not in globals() or 'val_df' not in globals() or 'test_df' not in globals():
    import pickle
    import os
    splits_save_path = 'dataset_splits.pkl'
    if os.path.exists(splits_save_path):
        print(f"Loading dataset splits from {splits_save_path}...")
        with open(splits_save_path, 'rb') as f:
            splits_data = pickle.load(f)
        train_df = splits_data['train_df']
        val_df = splits_data['val_df']
        test_df = splits_data['test_df']
        print(f"Loaded dataset splits: Train={len(train_df)}, Val={len(val_df)}, Test={len(test_df)}")
    else:
        raise NameError(
            f"Dataset splits not found and '{splits_save_path}' doesn't exist!\n"
            "   Please run data preparation steps (Steps 3-4) first."
        )

# Create datasets
train_dataset = VanGoghDataset(train_df, transform=train_transform)
val_dataset = VanGoghDataset(val_df, transform=eval_transform)
test_dataset = VanGoghDataset(test_df, transform=eval_transform)

print(f"Datasets: Train={len(train_dataset)}, Val={len(val_dataset)}, Test={len(test_dataset)}")


## 5. Training Functions


In [ ]:
# Training and evaluation functions
def train_one_epoch(model, loader, optimizer, criterion, device, max_grad_norm=1.0):
    """Train for one epoch: forward pass, compute loss, backward pass, update weights"""
    model.train()
    total_loss = 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        # Gradient clipping for training stability
        if max_grad_norm > 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        total_loss += loss.item()
        if wandb.run is not None:
            wandb.log({"batch_loss": loss.item()})
    return total_loss / len(loader)

@torch.no_grad()
def eval_one_epoch(model, loader, criterion, device):
    """Evaluate: forward pass only, compute loss and accuracy"""
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        total_loss += loss.item()
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    return total_loss / len(loader), correct / total

print("Training functions defined")


## 6. Hyperparameter Tuning with Optuna


In [ ]:
# Create model: supports both VGG19 and AlexNet (project requirement)
def create_model(model_name='VGG19', freeze_features=True, dropout=0.5):
    """
    Create model with binary classifier.
    Smart Fine-tuning Strategy:
    - VGG19: Unfreeze last 8 layers (top blocks) when freeze_features=False
    - AlexNet: Unfreeze last 2 layers (top layers) when freeze_features=False
    """
    if model_name == 'VGG19':
        model = models.vgg19(weights='IMAGENET1K_V1')
        layers_to_unfreeze_count = 8  # Unfreeze roughly half the model (top blocks)
    elif model_name == 'AlexNet':
        model = models.alexnet(weights='IMAGENET1K_V1')
        layers_to_unfreeze_count = 2  # Unfreeze only the very top layers
    else:
        raise ValueError(f"Unknown model: {model_name}")
    
    # 1. Freeze everything by default
    for param in model.features.parameters():
        param.requires_grad = False
        
    # 2. Smart Unfreeze (if requested)
    if not freeze_features:
        # Find all layers that have weights (trainable layers)
        trainable_layers = [layer for layer in model.features if hasattr(layer, 'weight')]
        
        # Unfreeze only the last X layers based on model type
        layers_to_unfreeze = trainable_layers[-layers_to_unfreeze_count:]
        
        for layer in layers_to_unfreeze:
            for param in layer.parameters():
                param.requires_grad = True
                
    # 3. Replace Classifier Head (Always trainable)
    if model_name == 'VGG19':
        num_features = model.classifier[6].in_features
        model.classifier[6] = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(num_features, 2)
        )
    elif model_name == 'AlexNet':
        num_features = model.classifier[6].in_features
        model.classifier[6] = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(num_features, 2)
        )
    
    return model.to(device)

def objective(trial):
    """Optuna objective: try hyperparameters, train model, return validation accuracy"""
    # Optuna suggests hyperparameters
    model_name = trial.suggest_categorical("model_name", ["VGG19", "AlexNet"])  # Project requirement: both models
    lr = trial.suggest_float("lr", 1e-5, 5e-4, log=True)  # Reduced max LR for stability (was 1e-2)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "SGD", "AdamW"])
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32, 64])  # Added 8 for smaller GPUs
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-2, log=True)
    momentum = trial.suggest_float("momentum", 0.8, 0.99) if optimizer_name == "SGD" else 0.0
    freeze_features = trial.suggest_categorical("freeze_features", [True, False])
    dropout = trial.suggest_float("dropout", 0.0, 0.7)  # Added dropout search

    # Create DataLoaders, model, optimizer
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)  # Like Nir
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    run = wandb.init(
        project="VanGogh_Classifier",
        name=f"trial_{trial.number}_{model_name}",
        config={"model_name": model_name, "lr": lr, "optimizer": optimizer_name, "batch_size": batch_size,
                "weight_decay": weight_decay, "momentum": momentum,
                "freeze_features": freeze_features, "dropout": dropout},
        reinit=True
    )
    
    # Define epoch as step metric to ensure X-axis starts at 0 for each trial
    run.define_metric("epoch", hidden=True)
    run.define_metric("train_loss", step_metric="epoch")
    run.define_metric("val_loss", step_metric="epoch")
    run.define_metric("val_acc", step_metric="epoch")
    run.define_metric("best_val_acc", step_metric="epoch")

    model = create_model(model_name=model_name, freeze_features=freeze_features, dropout=dropout)
    criterion = nn.CrossEntropyLoss()
    trainable_params = filter(lambda p: p.requires_grad, model.parameters())

    if optimizer_name == "SGD":
        optimizer = torch.optim.SGD(trainable_params, lr=lr, momentum=momentum, weight_decay=weight_decay)
    elif optimizer_name == "AdamW":
        optimizer = torch.optim.AdamW(trainable_params, lr=lr, weight_decay=weight_decay)
    else:
        optimizer = torch.optim.Adam(trainable_params, lr=lr, weight_decay=weight_decay)

    # Train for a few epochs (quick evaluation for hyperparameter search)
    num_epochs = 3  # Reduced for faster trials (30-60 min total search time)
    best_val_acc = 0.0
    for epoch in range(num_epochs):
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
        val_loss, val_acc = eval_one_epoch(model, val_loader, criterion, device)
        best_val_acc = max(best_val_acc, val_acc)
        wandb.log({"epoch": epoch, "train_loss": train_loss, "val_loss": val_loss,
                   "val_acc": val_acc, "best_val_acc": best_val_acc}, step=epoch)
        trial.report(val_acc, epoch)
        if trial.should_prune():  # Stop bad trials early
            run.finish()
            raise optuna.exceptions.TrialPruned()
    run.finish()
    return best_val_acc

print("Optuna objective function defined (supports VGG19 and AlexNet)")


In [ ]:
# Run hyperparameter search with Optuna
# Project requirement: must take at least 30 minutes, max 60 minutes
import optuna  # Ensure optuna is imported (in case cells run out of order)
import time

print("Starting hyperparameter search...")
print("="*60)

study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.MedianPruner(n_startup_trials=3, n_warmup_steps=2)
)

# Track start time
start_time = time.time()

# Run optimization (max 60 minutes - will stop at timeout)
study.optimize(objective, n_trials=10, timeout=3600, show_progress_bar=True)  # 10 trials for 30-60 min window

# Calculate elapsed time
elapsed_time = time.time() - start_time
elapsed_minutes = elapsed_time / 60

print("\n" + "="*60)
print("HYPERPARAMETER SEARCH COMPLETE!")
print("="*60)
print(f"\nTime taken: {elapsed_minutes:.2f} minutes ({elapsed_time:.0f} seconds)")
print(f"Completed trials: {len(study.trials)} / 10")

# Check project requirements
if elapsed_minutes < 30:
    print(f"\nWARNING: Search took only {elapsed_minutes:.2f} minutes!")
    print("   Project requires at least 30 minutes. Consider increasing n_trials.")
elif elapsed_minutes > 60:
    print(f"\nWARNING: Search took {elapsed_minutes:.2f} minutes (exceeded 60 min limit)")
else:
    print(f"\nTime requirement met: {elapsed_minutes:.2f} minutes (30-60 min range)")

print(f"\nBest validation accuracy: {study.best_value:.4f}")
print(f"\nBest hyperparameters:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

# Save study results for later use (so step 7 can run independently)
import pickle
study_save_path = 'optuna_study_results.pkl'
with open(study_save_path, 'wb') as f:
    pickle.dump({
        'best_params': study.best_params,
        'best_value': study.best_value,
        'n_trials': len(study.trials)
    }, f)
print(f"\nStudy results saved to '{study_save_path}' (can skip step 6 next time)")


## 7. Train Final Model


In [ ]:
# GPU Utilization Monitoring (Required for appendix)
# Run nvidia-smi to show GPU utilization and memory usage
import subprocess
import sys

print("="*60)
print("GPU UTILIZATION (nvidia-smi)")
print("="*60)
try:
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True, timeout=5)
    print(result.stdout)
    if result.stderr:
        print("Stderr:", result.stderr, file=sys.stderr)
except FileNotFoundError:
    print("nvidia-smi not found. This is expected if running on CPU or in some environments.")
    print("   GPU monitoring will be skipped.")
except subprocess.TimeoutExpired:
    print("nvidia-smi timed out.")
except Exception as e:
    print(f"Could not run nvidia-smi: {e}")
print("="*60)

In [ ]:
# Train final model with best hyperparameters
import pickle
import os

# Load best_params if study doesn't exist (in case step 6 wasn't run)
if 'study' not in globals() or study is None:
    study_save_path = 'optuna_study_results.pkl'
    if os.path.exists(study_save_path):
        print(f"Loading hyperparameters from {study_save_path}...")
        with open(study_save_path, 'rb') as f:
            study_data = pickle.load(f)
        best_params = study_data['best_params']
        print(f"Loaded hyperparameters (Best val acc: {study_data['best_value']:.4f})")
    else:
        raise NameError(
            f"'study' not found and '{study_save_path}' doesn't exist!\n"
            "   Please run Step 6 (Hyperparameter Tuning) first."
        )
else:
    best_params = study.best_params

print("\nTraining final model with best parameters:")
for k, v in best_params.items():
    print(f"  {k}: {v}")

# Anti-overfitting improvements: increase dropout and weight decay
# Optuna found dropout ~0.29, but we have overfitting - increase to 0.4 for better regularization
original_dropout = best_params.get('dropout', 0.5)
improved_dropout = max(0.4, min(0.5, original_dropout + 0.1))  # Increase by 0.1, cap at 0.5
print(f"\nAnti-overfitting adjustments:")
print(f"   Dropout: {original_dropout:.4f} -> {improved_dropout:.4f}")

original_weight_decay = best_params.get('weight_decay', 1e-4)
improved_weight_decay = original_weight_decay * 1.25  # Increase by 25% for more regularization
print(f"   Weight decay: {original_weight_decay:.6f} → {improved_weight_decay:.6f}")

# Setup: DataLoaders, model, optimizer, scheduler
# Stability improvement: Manually set batch_size to 32 for final training (as per instructions)
# This reduces 'zigzag' effect in loss curves, regardless of Optuna suggestion
final_batch_size = 32  # Fixed batch size for stability (use 16 if memory is an issue)
print(f"\nBatch size: Optuna suggested {best_params.get('batch_size', 'N/A')}, using {final_batch_size} for stability")
final_train_loader = DataLoader(train_dataset, batch_size=final_batch_size, shuffle=True, num_workers=2)
final_val_loader = DataLoader(val_dataset, batch_size=final_batch_size, shuffle=False, num_workers=2)

final_model = create_model(
    model_name=best_params.get('model_name', 'VGG19'),
    freeze_features=best_params.get('freeze_features', False),
    dropout=improved_dropout  # Use improved dropout
)
criterion = nn.CrossEntropyLoss()
trainable_params = filter(lambda p: p.requires_grad, final_model.parameters())

if best_params['optimizer'] == "SGD":
    final_optimizer = torch.optim.SGD(trainable_params, lr=best_params['lr'],
                                       momentum=best_params.get('momentum', 0.9),
                                       weight_decay=improved_weight_decay)  # Use improved weight decay
elif best_params['optimizer'] == "AdamW":
    final_optimizer = torch.optim.AdamW(trainable_params, lr=best_params['lr'],
                                         weight_decay=improved_weight_decay)  # Use improved weight decay
else:
    final_optimizer = torch.optim.Adam(trainable_params, lr=best_params['lr'],
                                        weight_decay=improved_weight_decay)  # Use improved weight decay

# Improved learning rate scheduler: ReduceLROnPlateau with better settings
# Use min_lr to prevent LR from going too low, and smaller factor for smoother decay
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    final_optimizer, mode='max', factor=0.5, patience=2, 
    min_lr=1e-6, threshold=0.001
)

# Training loop with early stopping
run = wandb.init(project="VanGogh_Classifier", name="final_model_training",
                 config={
                     **best_params, 
                     "training_type": "final", 
                     "gradient_clipping": True, 
                     "final_batch_size": final_batch_size,
                     "freeze_features": True,  # Log the actual parameter used
                     "improved_dropout": improved_dropout,
                     "improved_weight_decay": improved_weight_decay
                 }, reinit=True)

# Define epoch as step metric to ensure X-axis starts at 0 for final training
run.define_metric("epoch", hidden=True)
run.define_metric("train_loss", step_metric="epoch")
run.define_metric("val_loss", step_metric="epoch")
run.define_metric("val_acc", step_metric="epoch")
run.define_metric("lr", step_metric="epoch")

num_epochs = 15  # Reduced for time efficiency
best_val_acc = 0.0
best_model_state = None
patience = 5  # Increased early stopping patience (was 3) to allow more recovery
epochs_without_improvement = 0
train_losses, val_losses, val_accuracies = [], [], []

print("\nStarting training...")
print("="*60)
print("Improvements applied:")
print("   - Gradient clipping (max_norm=1.0) for stability")
print("   - Increased early stopping patience (3->5 epochs)")
print("   - Improved LR scheduler (min_lr, threshold)")
print("   - Increased dropout for better regularization")
print("   - Increased weight decay for more L2 regularization")
print("="*60)

for epoch in range(num_epochs):
    train_loss = train_one_epoch(final_model, final_train_loader, final_optimizer, criterion, device, max_grad_norm=1.0)
    val_loss, val_acc = eval_one_epoch(final_model, final_val_loader, criterion, device)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    scheduler.step(val_acc)
    current_lr = final_optimizer.param_groups[0]['lr']
    
    # Log to wandb with all parameters (graphs will be displayed in wandb)
    wandb.log({
        "epoch": epoch, 
        "train_loss": train_loss, 
        "val_loss": val_loss,
        "val_acc": val_acc, 
        "lr": current_lr,
        "freeze_features": True,  # Log the parameter
        "dropout": improved_dropout,
        "weight_decay": improved_weight_decay
    }, step=epoch)

    print(f"Epoch {epoch+1:02d}/{num_epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f} | LR: {current_lr:.2e}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_state = {k: v.cpu().clone() for k, v in final_model.state_dict().items()}
        epochs_without_improvement = 0
        print(f"  New best model! (Val Acc: {best_val_acc:.4f})")
    else:
        epochs_without_improvement += 1

    if epochs_without_improvement >= patience:
        print(f"\nEarly stopping after {epoch+1} epochs")
        break

final_model.load_state_dict(best_model_state)
print(f"\n{'='*60}")
print(f"Training complete! Best validation accuracy: {best_val_acc:.4f}")

# Save model and training history for later use (so you don't need to retrain)
torch.save({
    'model_state_dict': best_model_state, 
    'best_params': best_params,
    'best_val_acc': best_val_acc,
    'train_losses': train_losses,
    'val_losses': val_losses,
    'val_accuracies': val_accuracies
}, 'best_vangogh_classifier.pth')
print("Model and training history saved to 'best_vangogh_classifier.pth'")

run.finish()


## 8. Visualizations


In [ ]:
# Plot training curves
import matplotlib.pyplot as plt  # Ensure matplotlib is imported
import os
import torch  # Import torch for loading checkpoint

# Load training history if variables don't exist (in case kernel was restarted)
if 'train_losses' not in globals() or 'val_losses' not in globals() or 'val_accuracies' not in globals() or 'best_val_acc' not in globals():
    checkpoint_path = 'best_vangogh_classifier.pth'
    if os.path.exists(checkpoint_path):
        print(f"Loading training history from {checkpoint_path}...")
        checkpoint = torch.load(checkpoint_path, map_location='cpu')
        train_losses = checkpoint.get('train_losses', [])
        val_losses = checkpoint.get('val_losses', [])
        val_accuracies = checkpoint.get('val_accuracies', [])
        best_val_acc = checkpoint.get('best_val_acc', 0.0)
        print(f"Loaded training history: {len(train_losses)} epochs, Best Val Acc: {best_val_acc:.4f}")
    else:
        raise NameError(
            f"Training variables not found and checkpoint file '{checkpoint_path}' doesn't exist!\n"
            "   Please run Step 7 (Train Final Model) first before plotting."
        )

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(train_losses, label='Train Loss', color='#1f77b4', linewidth=2)
axes[0].plot(val_losses, label='Validation Loss', color='#ff7f0e', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training and Validation Loss', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(val_accuracies, label='Validation Accuracy', color='#2ca02c', linewidth=2, marker='o')
axes[1].axhline(y=best_val_acc, color='r', linestyle='--', label=f'Best: {best_val_acc:.4f}')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Validation Accuracy', fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_curves.png', dpi=150)
plt.show()
print("Saved: training_curves.png")


## 9. Test Set Evaluation


In [ ]:
# Evaluate on test set (final performance metric)
import torch
import os

# Load best_params if not available (needed for batch_size)
if 'best_params' not in globals():
    checkpoint_path = 'best_vangogh_classifier.pth'
    if os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path, map_location='cpu')
        best_params = checkpoint.get('best_params')
        if best_params:
            print(f"Loaded best_params from checkpoint")
    if 'best_params' not in globals() or best_params is None:
        import pickle
        study_save_path = 'optuna_study_results.pkl'
        if os.path.exists(study_save_path):
            with open(study_save_path, 'rb') as f:
                study_data = pickle.load(f)
            best_params = study_data['best_params']
            print(f"Loaded best_params from study file")
        else:
            raise NameError(
                "Cannot find best_params!\n"
                "   Please run Step 6 (Hyperparameter Tuning) or Step 7 (Train Final Model) first."
            )

# Check if final_model exists (step 7 must be run for evaluation)
if 'final_model' not in globals():
    raise NameError(
        "'final_model' not found!\n"
        "   Please run Step 7 (Train Final Model) first before evaluation.\n"
        "   The model needs to be created and trained in step 7."
    )

test_loader = DataLoader(test_dataset, batch_size=best_params['batch_size'], shuffle=False, num_workers=2)

final_model.eval()
all_preds, all_labels, all_probs = [], [], []

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Evaluating"):
        images, labels = images.to(device), labels.to(device)
        outputs = final_model(images)
        probs = torch.softmax(outputs, dim=1)
        preds = outputs.argmax(dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        all_probs.extend(probs[:, 1].cpu().numpy())

all_preds, all_labels, all_probs = np.array(all_preds), np.array(all_labels), np.array(all_probs)

# Calculate metrics
test_accuracy = accuracy_score(all_labels, all_preds)
test_precision = precision_score(all_labels, all_preds)
test_recall = recall_score(all_labels, all_preds)
test_f1 = f1_score(all_labels, all_preds)
test_auc = roc_auc_score(all_labels, all_probs)

print("="*60)
print("TEST SET RESULTS")
print("="*60)
print(f"\nMetrics:")
print(f"   Accuracy:  {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print(f"   Precision: {test_precision:.4f}")
print(f"   Recall:    {test_recall:.4f}")
print(f"   F1-Score:  {test_f1:.4f}")
print(f"   AUC-ROC:   {test_auc:.4f}")

print(f"\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=['Not Van Gogh', 'Van Gogh']))


In [ ]:
# Confusion Matrix and ROC Curve
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

cm = confusion_matrix(all_labels, all_preds)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Not Van Gogh', 'Van Gogh'],
            yticklabels=['Not Van Gogh', 'Van Gogh'], annot_kws={'size': 14})
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')
axes[0].set_title('Confusion Matrix', fontweight='bold')

fpr, tpr, _ = roc_curve(all_labels, all_probs)
axes[1].plot(fpr, tpr, color='#1f77b4', linewidth=2, label=f'ROC (AUC = {test_auc:.4f})')
axes[1].plot([0, 1], [0, 1], color='gray', linestyle='--')
axes[1].fill_between(fpr, tpr, alpha=0.2, color='#1f77b4')
axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate')
axes[1].set_title('ROC Curve', fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('confusion_matrix_roc.png', dpi=150)
plt.show()
print("Saved: confusion_matrix_roc.png")


In [ ]:
# Visualize False Positives (images predicted as Van Gogh but are not)
# This is important for analysis in Part 3
import matplotlib.pyplot as plt
from PIL import Image

# Find False Positives
false_positives = []
for idx, (pred, label, prob) in enumerate(zip(all_preds, all_labels, all_probs)):
    if pred == 1 and label == 0:  # Predicted Van Gogh but is not
        false_positives.append((idx, prob))

# Sort by confidence (highest probability first)
false_positives.sort(key=lambda x: x[1], reverse=True)

# Display top False Positives
num_to_show = min(12, len(false_positives))
if num_to_show > 0:
    fig, axes = plt.subplots(3, 4, figsize=(16, 12))
    fig.suptitle('False Positives: Predicted as Van Gogh (but are not)', fontsize=16, fontweight='bold')
    
    for i, (idx, prob) in enumerate(false_positives[:num_to_show]):
        row = i // 4
        col = i % 4
        
        # Get image path from test dataset
        image_path = test_df.iloc[idx]['filepath']
        image = Image.open(image_path)
        
        axes[row, col].imshow(image)
        axes[row, col].set_title(f'Conf: {prob:.3f}\n{test_df.iloc[idx]["artist"]}', fontsize=10)
        axes[row, col].axis('off')
    
    # Hide empty subplots
    for i in range(num_to_show, 12):
        row = i // 4
        col = i % 4
        axes[row, col].axis('off')
    
    plt.tight_layout()
    plt.savefig('false_positives.png', dpi=150)
    plt.show()
    print(f"Saved: false_positives.png")
    print(f"\nTotal False Positives: {len(false_positives)} / {len(all_preds)}")
else:
    print("No False Positives found!")


## 10. Final Summary


In [ ]:
# Final Summary
print("="*60)
print("PART A - VAN GOGH CLASSIFIER - SUMMARY")
print("="*60)

print("\nDataset:")
print(f"   Total: {len(df)} | Van Gogh: {df['is_van_gogh'].sum()} | Other: {len(df)-df['is_van_gogh'].sum()}")
print(f"   Split: 70% train / 15% val / 15% test")

print("\nBest Hyperparameters:")
for k, v in best_params.items():
    print(f"   {k}: {v}")

print("\nPerformance:")
print(f"   Best Val Acc:  {best_val_acc:.4f} ({best_val_acc*100:.2f}%)")
print(f"   Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print(f"   Test F1-Score: {test_f1:.4f}")
print(f"   Test AUC-ROC:  {test_auc:.4f}")

print("\nSaved Files:")
print("   best_vangogh_classifier.pth")
print("   training_curves.png")
print("   confusion_matrix_roc.png")

print("\nPart A Complete!")
print("="*60)
